# Preminary operations

Let us start by importing the relevant packages

In [ ]:
import gcp.bigquery as bq
import pandas as pd
import matplotlib.pylab as plt
import numpy as np
import seaborn as sns

## Let us explore the dataset

In [ ]:
%%sql 
SELECT * FROM [google.com:ddm-connectors-demo:aw_dataset.__TABLES__];

## Lest us explore the table KEYWORDS_PERFORMANCE_ANALYTICS

In [ ]:
%bigquery schema --table google.com:ddm-connectors-demo:aw_dataset.KEYWORDS_PERFORMANCE_ANALYTICS

## Let us see how the data look like

In [ ]:
%bigquery sample --table google.com:ddm-connectors-demo:aw_dataset.KEYWORDS_PERFORMANCE_ANALYTICS --count 5

# Let us start checking the monthly performance on Volagratis in Italy

In [ ]:
%sql --module monthly_keywords_performance
SELECT
  AccountDescriptiveName,
  CampaignName,
  AdGroupName,
  Criteria,
  KeywordMatchType,
  COUNT(Month) as MonthsWithImpressions,
  AVG(Impressions) as Impressions,
  AVG(ImpressionAssistedConversions) as ImpressionAssistedConversions,
  AVG(Clicks) as Clicks,
  AVG(ClickAssistedConversions) as ClickAssistedConversions,  
  AVG(CTR) as CTR,
  AVG(QualityScore) as QualityScore, 
  AVG(CostEUR) as CostEUR,
  AVG(AllConversions) as AllConversions,
  AVG(Conversions) as Conversions,    
  AVG(CostPerAllConversion) as CostPerAllConversion,
  AVG(CostPerConversion) as CostPerConversion,  
  #Analytics Metrics
  AVG(BounceRate) as BounceRate,
  AVG(AveragePosition) as AveragePosition,
  AVG(AveragePageviews) as AveragePageviews,
  AVG(AverageTimeOnSite) as AverageTimeOnSite,
  AVG(PercentNewVisitors) as PercentNewVisitors,
FROM
(
SELECT
  Month,
  AccountDescriptiveName,
  CampaignName,
  AdGroupName,
  Criteria,
  KeywordMatchType,
  SUM(Impressions) as Impressions,
  SUM(ImpressionAssistedConversions) as ImpressionAssistedConversions,
  SUM(Clicks) as Clicks,
  SUM(Clicks)/SUM(Impressions) as CTR,
  SUM(IF(ClickAssistedConversionsOverLastClickConversions = 0, 0,
  CASE
    WHEN Conversions > 0 OR AllConversions > 0 THEN IF(AllConversions>Conversions,AllConversions,Conversions)*ClickAssistedConversionsOverLastClickConversions
    WHEN ImpressionAssistedConversions > 0 THEN ImpressionAssistedConversions*ClickAssistedConversionsOverLastClickConversions
    ELSE 1.0
  END
  )) as ClickAssistedConversions,
  
  AVG(QualityScore) as QualityScore,
  
  SUM(IF(AccountCurrencyCode = 'GBP', Cost*1.37, Cost*1.0))/1000000 as CostEUR,
  SUM(AllConversions) as AllConversions,
  SUM(Conversions) as Conversions,  
  
  SUM(IF(AccountCurrencyCode = 'GBP', Cost*1.37, Cost*1.0))/1000000/SUM(AllConversions) as CostPerAllConversion,
  SUM(IF(AccountCurrencyCode = 'GBP', Cost*1.37, Cost*1.0))/1000000/SUM(Conversions) as CostPerConversion,
  
  #Analytics Metrics
  SUM(FLOAT(RTRIM(BounceRate,'%'))/100.0 * Clicks)/SUM(Clicks) as BounceRate,
  SUM(AveragePosition * Impressions)/SUM(Impressions) as AveragePosition,
  SUM(AveragePageviews * Clicks)/SUM(Clicks) as AveragePageviews,
  SUM(AverageTimeOnSite * Clicks)/SUM(Clicks) as AverageTimeOnSite,
  SUM(FLOAT(RTRIM(PercentNewVisitors,'%'))/100.0 * Clicks)/SUM(Clicks) as PercentNewVisitors,

FROM
  [google.com:ddm-connectors-demo:aw_dataset.KEYWORDS_PERFORMANCE_ANALYTICS]
WHERE TRUE
AND NOT IsNegative
AND AdNetworkType2 = 'Google search'
AND Impressions > 0 
AND Year = 2016
AND AccountDescriptiveName IN ('Account Primario Volagratis','Volagratis Tratte Italia','Volagratis')
GROUP BY 1,2,3,4,5,6)
GROUP BY 1,2,3,4,5

In [ ]:
df = bq.Query(monthly_keywords_performance).to_dataframe()

First of all let us identify the Brand Keywords assuming that 
- they have at least 10 impressions a month
- their CTR is higher the "normal"

In [ ]:
df[df['Impressions']>10].hist('CTR',bins=np.arange(101)/100.0,
                              normed=True,cumulative=1,histtype='step')

In [ ]:
df[(df['AdGroupName'] == 'Brand')&(df['Impressions']>10)].hist('CTR',bins=np.arange(101)/100.0,
                                                              cumulative=1,normed=1,
                                                               histtype='step')

In [ ]:
df[(df['AdGroupName'] == 'Brand')&
   (df['CTR']<0.4)&
   (df['Impressions']>10)].sort_values(by='CostEUR',ascending=False)[['AccountDescriptiveName',
                                                                                        'CampaignName',
                                                                                        'AdGroupName',
                                                                                        'Criteria',
                                                                                        'KeywordMatchType',
                                                                                        'CTR',
                                                                                        'Impressions',
                                                                                        'CostEUR']]

In [ ]:
df[(df['Impressions']>10) & (df['CTR']>0.4)].sort_values(by='CostEUR',ascending=False)[['AccountDescriptiveName',
                                                                                        'CampaignName',
                                                                                        'AdGroupName',
                                                                                        'Criteria',
                                                                                        'KeywordMatchType',
                                                                                        'CTR',
                                                                                        'Impressions',
                                                                                        'CostEUR']]

In [ ]:
df['KeywordType'] = df.apply(lambda x: 'Gold' if 
                             (x['Impressions']>10 and x['CTR']>0.4) 
                             else 'Standard', axis=1)


Let us start assessing the performance of the keywords from the AllConversions

In [ ]:
df.groupby(by=(df['AllConversions'] > 0))[['AllConversions','ClickAssistedConversions',
                                           'ImpressionAssistedConversions','CostEUR',
                                           'Impressions','Clicks',
                                           'Criteria']].aggregate({'AllConversions':sum,
                                                                   'ClickAssistedConversions':sum,
                                                                   'ImpressionAssistedConversions':sum,
                                                                   'CostEUR':sum,
                                                                   'Impressions':sum,
                                                                   'Clicks':sum,
                                                                   'Criteria':'count'})

In [ ]:
df[df['AllConversions'] > 0].hist('Conversions',bins=100)

In [ ]:
axes = df[df['AllConversions'] > 0].hist('AllConversions',bins=np.logspace(0,4),log=True,by='KeywordType')
axes[0].set_xscale('log')
axes[1].set_xscale('log')

In [ ]:
axes = df[df['AllConversions'] > 0].hist('AllConversions',bins=np.logspace(0,4),by='KeywordType',
              cumulative=1,normed=True,histtype='step')
axes[0].set_xscale('log')
axes[1].set_xscale('log')

In [ ]:
df[df['AllConversions'] > 0].hist('AllConversions',bins=np.logspace(0,4),
              cumulative=1,histtype='step')
plt.gca().set_xscale('log')

In [ ]:
def setGroup(x):
  if x['AllConversions'] > 0:
    return 'Converter'
  else:
    return 'Other'
  
def setSubGroup(x):
  if x['AllConversions'] > 99:
    return 'Converter - High'
  elif x['AllConversions'] > 9:
    return 'Converter - Mid'
  elif x['AllConversions'] > 2:
    return 'Converter - Low'
  elif x['AllConversions'] > 0:
    return 'Converter - Occasional'
  else:
    return 'Other'
  
df['Group'] = df.apply(setGroup,axis=1)
df['SubGroup'] = df.apply(setSubGroup,axis=1)

In [ ]:
g = sns.pairplot(data=df[df['Group'] == 'Converter'][[u'SubGroup','CTR',
                                                  'CostPerAllConversion',
                            u'QualityScore',u'AveragePosition',                          
                            u'BounceRate', u'AveragePageviews',
                            u'AverageTimeOnSite',u'PercentNewVisitors']],hue=u'SubGroup')

In [ ]:
g = sns.pairplot(data=df[df['Group'] == 'Converter'][[u'SubGroup',                         
                            u'BounceRate', u'AveragePageviews',
                            u'AverageTimeOnSite',u'PercentNewVisitors']],hue=u'SubGroup')

In [ ]:
g = sns.pairplot(data=df[df['Group'] == 'Converter'][[u'SubGroup','CTR',
                                                  'CostPerAllConversion',
                            u'QualityScore',u'AveragePosition',                          
                            u'BounceRate',u'PercentNewVisitors']],hue=u'SubGroup')

Can we move away from LastClick?

In [ ]:
df.groupby(by='SubGroup')[['AllConversions','ClickAssistedConversions',
                        'ImpressionAssistedConversions',
                        'CostEUR','Impressions','Clicks',
                        'Criteria']].aggregate({'AllConversions':sum,
                                                'ClickAssistedConversions':sum,
                                                'ImpressionAssistedConversions':sum,
                                                'CostEUR':sum,
                                                'Impressions':sum,
                                                'Clicks':sum,
                                                'Criteria':'count'})

In [ ]:
df[df['Group'] == 'Other'].groupby(by=(df[df['Group'] == 'Other']['ClickAssistedConversions'] +
                                       df[df['Group'] == 'Other']['ImpressionAssistedConversions']>0))[
  ['ClickAssistedConversions','ImpressionAssistedConversions','CostEUR',
   'Impressions','Clicks','Criteria']].aggregate({'ClickAssistedConversions':sum,
                                                  'ImpressionAssistedConversions':sum,
                                                  'CostEUR':sum,
                                                  'Impressions':sum,
                                                  'Clicks':sum,
                                                  'Criteria':'count'})

In [ ]:
df[(df['Group'] == 'Other')&(df['ClickAssistedConversions'] > 0)].hist('ClickAssistedConversions',
                                                                       bins=100,
                                                                       cumulative=1,
                                                                       normed=True,
                                                                       histtype='step')

In [ ]:
df[(df['Group'] == 'Other')&(df['ImpressionAssistedConversions'] > 0)].hist('ImpressionAssistedConversions',
                                                                       bins=100,
                                                                       cumulative=1,
                                                                       normed=True,
                                                                       histtype='step')